# LaptopでCustom Vision Serviceを動かしてみる

- Main reference: https://docs.microsoft.com/ja-jp/azure/cognitive-services/custom-vision-service/export-model-python
    - Export `model.py`, `label.txt` from [custom vision services](https://www.customvision.ai/)
    - Put test images at `./test/` directory 
    - no GPU required


In [2]:
## Library import
import tensorflow as tf
import os

from PIL import Image
import numpy as np
import cv2
import datetime

## Some tools

In [3]:
def convert_to_opencv(image):
    # RGB -> BGR conversion is performed as well.
    r,g,b = np.array(image).T
    opencv_image = np.array([b,g,r]).transpose()
    return opencv_image

def crop_center(img,cropx,cropy):
    h, w = img.shape[:2]
    startx = w//2-(cropx//2)
    starty = h//2-(cropy//2)
    return img[starty:starty+cropy, startx:startx+cropx]

def resize_down_to_1600_max_dim(image):
    h, w = image.shape[:2]
    if (h < 1600 and w < 1600):
        return image

    new_size = (1600 * w // h, 1600) if (h > w) else (1600, 1600 * h // w)
    return cv2.resize(image, new_size, interpolation = cv2.INTER_LINEAR)

def resize_to_256_square(image):
    h, w = image.shape[:2]
    return cv2.resize(image, (256, 256), interpolation = cv2.INTER_LINEAR)

def update_orientation(image):
    exif_orientation_tag = 0x0112
    if hasattr(image, '_getexif'):
        exif = image._getexif()
        if (exif != None and exif_orientation_tag in exif):
            orientation = exif.get(exif_orientation_tag, 1)
            # orientation is 1 based, shift to zero based and flip/transpose based on 0-based values
            orientation -= 1
            if orientation >= 4:
                image = image.transpose(Image.TRANSPOSE)
            if orientation == 2 or orientation == 3 or orientation == 6 or orientation == 7:
                image = image.transpose(Image.FLIP_TOP_BOTTOM)
            if orientation == 1 or orientation == 2 or orientation == 5 or orientation == 6:
                image = image.transpose(Image.FLIP_LEFT_RIGHT)
    return image

In [4]:
## Some setting
WKDIR = "./test/"
#IMAGE_FILE = "IMG_0310.JPG"
#IMAGE_FILE = "IMG_0318.JPG"
#IMAGE_FILE = "IMG_0453.JPG"
#IMAGE_FILE = "IMG_0455.JPG"
IMAGE_FILE = "IMG_0459.JPG"
#IMAGE_FILE = "IMG_0543.JPG"

In [5]:
graph_def = tf.GraphDef()
labels = []

# These are set to the default names from exported models, update as needed.
# exportモデルとlabelを読み込む
filename = "model.pb"
labels_filename = "labels.txt"

# Import the TF graph
with tf.gfile.GFile(filename, 'rb') as f:
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')

# labelを読み込む
with open(labels_filename, 'rt') as lf:
    for l in lf:
        labels.append(l.strip())

In [6]:
start_process = datetime.datetime.now()

# Load from a file
#imageFile = "<path to your image file>"
imageFile = WKDIR+IMAGE_FILE
image = Image.open(imageFile)

# Update orientation based on EXIF tags, if the file has orientation info.
image = update_orientation(image)

# Convert to OpenCV format
image = convert_to_opencv(image)

## Conversion of image

In [7]:
# If the image has either w or h greater than 1600 we resize it down respecting
# aspect ratio such that the largest dimension is 1600
image = resize_down_to_1600_max_dim(image)

# We next get the largest center square
h, w = image.shape[:2]
min_dim = min(w,h)
max_square_image = crop_center(image, min_dim, min_dim)

# Resize that square down to 256x256
augmented_image = resize_to_256_square(max_square_image)

In [8]:
# Get the input size of the model
with tf.Session() as sess:
    input_tensor_shape = sess.graph.get_tensor_by_name('Placeholder:0').shape.as_list()
network_input_size = input_tensor_shape[1]

# Crop the center for the specified network_input_Size
augmented_image = crop_center(augmented_image, network_input_size, network_input_size)

## TensorFlowによる推定

In [9]:
# These names are part of the model and cannot be changed.
output_layer = 'loss:0'
input_node = 'Placeholder:0'

with tf.Session() as sess:
    try:
        prob_tensor = sess.graph.get_tensor_by_name(output_layer)
        ## Prediction
        predictions, = sess.run(prob_tensor, {input_node: [augmented_image] })
    except KeyError:
        print ("Couldn't find classification output layer: " + output_layer + ".")
        print ("Verify this a model exported from an Object Detection project.")
        exit(-1)
    # Print the highest probability label
    highest_probability_index = np.argmax(predictions)
    print('Classified as: ' + labels[highest_probability_index])
    print()

    # Or you can print out all of the results mapping labels to probabilities.
    label_index = 0
    for p in predictions:
        truncated_probablity = np.float64(np.round(p,8))
        print (labels[label_index], truncated_probablity)
        label_index += 1
        
process_time = datetime.datetime.now() - start_process
print("Elapsed time is {}".format(process_time))

Classified as: food

city 9.350000254926272e-06
food 0.9985092282295227
forest 5.599999894911889e-07
Elapsed time is 0:00:00.936587
